In [ ]:
# 学历是否匹配、工作年限是否匹配

In [1]:
import sys
sys.path.append('..')
from feature_extract import person_feature,person_intent,work_history,certificate,project_history,job_table,diliver_current,match_current,get_data,get_k_fold_data,feature_process
from utils import load_data
from models.lgb_model import lgb_model_for_upload,lgb_model_for_offline_test
import numpy as np
import pandas as pd
import random
import sklearn
import os
path0=os.path.abspath(os.path.dirname(os.getcwd()))
path = path0 +'/data/info_dataset/'
path1 = path0+'/data/'

In [2]:
person =person_feature(path)

In [3]:
person

,求职者编号,性別,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,当前工作所在地,语言能力,专业特长
0,33291,男,15,大专,计算机应用技术,37,网络管理/信息安全管理,NaN,深圳市,普通话：精通 广东话：精通******语：听：良好，说：良好，读：良好，写：,"1、精通计算机软、硬件及网络维护，能迅速处理各种突发的计算机故障 2、熟悉局域网建设和管理,..."
1,2985277,男,12,大专,计算机应用技术,35,*公关/营销/业务类,文化体育行业,深圳市,普通话：精通 广东话：精通************语：听：精通，说：精通，读,..
2,2982066,女,10,大专,金融学（含保险学）,32,出纳,医药销售行业,南山区,普通话：精通 广东话：精通************语：听：精通，说：良好，读,****-****年获校三等奖学金****-****年获单项奖学金
3,3010866,男,10,中专,物理电子学,34,营销代表/销售顾问,珠宝玉石行业,深圳市,普通话：精通 广东话：良好************语：听：一般，说：不会，读,"我对塑胶厂里的运作流程非常熟悉,从业务,计划,生产,人事,采购,客户服务都有经验."
4,316816964,女,15,中专,学前教育学,34,小学教育/幼儿教育/保育,行业组织,福田区,NaN,1：拥有丰富的办公室工作经历，能独挡一面处理工作中的相关事情。2：熟练掌握Windows办公...
...,...,...,...,...,...,...,...,...,...,...,...
54774,4546928,男,14,大专,NaN,47,产品开发,娱乐业,深圳市,NaN,★经验描述： 曾在所述行业或职位上成功开发过银行柜员机、自动售货机、医疗设备、双电源开关、塑...
54775,6261517,男,26,大学本科,财务管理,48,财务总监CFO/总会计师,香料香精行业,广州市,文案及沟通能力较好，有较强的人际穿透能力。,一、在财务会计工作方面：1．能熟练处理工商企业全盘账税，熟练操作用友、金蝶、江苏速达等财务软...
54776,5642172,男,13,大专,NaN,32,项目管理,银行业,深圳市,NaN,1.熟练使用PRO-E/CAD、UG、OFFICE软件；2.对模具结构及成型工艺有很深的理解...
54777,5433634,男,13,博士研究生,管理科学与工程,39,总裁/总经理/CEO/主席/社长/总编,塑胶行业,深圳市,"具有较强的语听说读写能力。可流利阅读文资料,并能熟练收发文邮件",****公司化企业财务总监工作经验；精通现代企业财务管理并具有深厚的财务管理理论与实践经验；...


In [4]:
person["最近工作岗位"].value_counts()

会计            2608
国际贸易/涉外业务     2027
营销代表/销售顾问     1770
材料/部件采购管理     1312
*其它类          1064
              ... 
节目主持/DJ/播音       1
精算               1
企业管理咨询/服务        1
精细化工技术           1
文体经纪             1
Name: 最近工作岗位, Length: 297, dtype: int64

In [5]:
person["最近所在行业"].value_counts()

医疗器械行业    2608
物流业       2027
珠宝玉石行业    1770
其它        1064
互联网行业     1037
          ... 
园林景观行业       2
农林牧渔         1
林业           1
轻工业          1
生物医药行业       1
Name: 最近所在行业, Length: 123, dtype: int64

In [6]:
person["当前工作所在地"].value_counts()

深圳市    34475
宝安区     5563
龙岗区     2888
福田区     2493
南山区     2315
       ...  
澳门         1
济宁市        1
南宁市        1
国外         1
梧州市        1
Name: 当前工作所在地, Length: 70, dtype: int64

In [97]:
person[person["语言能力"].apply(lambda x: True if "普通话一般" in str(x) else False)]
# person["语言能力"].value_counts()#.sum()

,求职者编号,性別,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,当前工作所在地,语言能力,专业特长
6019,4744568,男,8,大学本科,公共行政管理,32,渠道管理,动漫产业,宝安区,普通话一般************广州话一般************语一般,1、熟悉人力资源管理流程，熟悉劳动法律及相关办事流程，擅长招聘、培训工作、员工关系处理；2、...
6866,4863748,男,10,大专,电子信息工程学,32,*其它类,其它,深圳市,普通话一般，语3级************熟悉电源安规测试流程与测试技巧,模拟电子基础知识扎实，熟悉电源安规测试流程与测试技巧，熟练使用与维护测试仪器。 对标...
6974,4557787,男,15,中专,NaN,33,司机,NaN,宝安区,"普通话一般,适应能力强.","电焊,氩弧焊,叉车的操作.维护.司机"
7092,4910822,男,12,大专,物理电子学,33,模具设计,租赁行业,深圳市,"语一般(三级左右),普通话一般","一、本人熟悉计算机各种绘图软件,如：AUTOCAD、PRO/E****、UG4.0、SOLI..."
7131,4909769,男,10,大专,软件工程,32,网络管理/信息安全管理,NaN,宝安区,语一般、普通话一般,在校期间主要修读：C++ C语言、汇编语言、JAVA、SQL****、VB、VF、等开发性...
...,...,...,...,...,...,...,...,...,...,...,...
52760,318529948,女,6,大专,通风、制冷与低温技术,28,绘图/建筑制图,仪器仪表行业,广东省,普通话一般、语过B级,熟练鸿业、天正软件制图，在老师指导下设计了六层办公大楼的中央空调的水系统中央空调设计，六层办...
52779,319097909,男,7,大专,NaN,29,电子技术,房地产开发行业,深圳市,普通话一般，语较差，粤语无,NaN
53637,320430902,女,7,大专,会计学,39,出纳,医药销售行业,龙岗区,粤语熟练、普通话一般、能听部分江门地区话及客家话。,本人****年取得珠算普通五级证书、****年于肇庆成人中专财会班毕业、****年取得...
53957,320607931,男,0,大学本科,测控技术与仪器,30,NaN,NaN,东莞市,普通话一般，语水平一般。,熟练使用C语言和汇编语言，具有软硬件开发能力。拥有****、AVR、msp****、ARM、...


In [122]:
person[person["语言能力"].apply(lambda x: True if ("二级" in str(x)) and ("4级" in str(x)) else False)]


,求职者编号,性別,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,当前工作所在地,语言能力,专业特长
1927,4589846,男,9,大学本科,机械制造及其自动化,32,机械设计,零售百货行业,深圳市,普通话 国家二级乙等水平************语国家4级水平,"专业理论知识扎实,熟练操作AUTOCAD,SOLIDWORKS及PRO/E等设计软件"
15930,5986258,男,10,大学本科,金融学（含保险学）,33,证券经纪/投资顾问,电力行业,深圳市,语4级************计算机二级,NaN
19467,6007463,女,0,大专,英语,29,国际贸易/涉外业务,物流业,深圳市,语4级************普通话二级乙等************日语和法语,英语听，说，读，写良好。能与外国人直接交谈，在我曾经的工作岗位上能有效地处理进口相关的业务。...
21267,6005362,女,10,大学本科,文秘,32,行政管理,互联网行业,深圳市,语4级************普通话二级,"1.精通Word,Excel等办公软件，熟悉ISO****、ISO****、OHSAS***..."
24388,5998629,男,0,大学本科,【电子科学与技术】,29,硬件开发,财务服务业,NaN,计算机水平 二级************语等级水平 4级,电路分析、模拟电路和数字电路基础扎实；有较好的C语言应用能力，了解Verilog HDL语言...
26644,6037953,女,9,大学本科,NaN,32,职业教育/培训/家教,科研机构,深圳市,全国语等级考试CET-4级证书；全国普通话测试二级乙；************,全国计算机等级考试一级证书；全国计算机等级考试二级证书；
26689,6055504,女,7,大学本科,英语,29,国际贸易/涉外业务,物流业,深圳市,获得全国普通话二级甲等证书。************通过了全国大学语4级，6级,本专业是电子商务（国际商务方向），通过这方面的学习，我掌握了一些国际贸易方面的基础知识。此外...
29873,6053546,女,10,大学本科,NaN,31,*金融/经济,能源/光电/电器行业,深圳市,大学语4级，较好的听说读写。普通话为省二级。,本人专业为国贸专业，对产品产品的采购以及对跟单船务有一定的经验。熟悉外贸出口货物的报关程序...
31634,6122790,女,7,大学本科,英语,29,*其它类,其它,深圳市,"大学语4、6级,专业语4级。国家普通话水平测试二级甲等，。熟练运用语进行",NaN
32323,6164413,男,0,中专,英语,31,*金融/经济,能源/光电/电器行业,宝安区,"语水平:口语流利,写作流畅,能与外商成功沟通(4级待考) 普通话:二级","熟练掌握所修課程：国际贸易，电子商务，网络营销，商贸法律，商务英语，英语函电，商贸谈判,职..."


In [21]:
pd.DataFrame(person["语言能力"].value_counts().rename_axis('语言能力').reset_index(name='总量')).to_csv(r'语言能力.csv')

In [22]:
language_skill =pd.DataFrame(person["语言能力"].value_counts().rename_axis('语言能力').reset_index(name='总量'))

In [23]:
language_skill

,语言能力,总量
0,普通话：精通 广东话：一般************语：听：一般，说：一般，读,511
1,良好,469
2,语四级,391
3,普通话,379
4,一般,356
...,...,...
34551,普通话标准，能良好的进行沟通与表达。语能力一般，能进行简单的交流。,1
34552,普通话熟练，目前已达到二乙水平，语水平为四级，粤语水平一般。,1
34553,语四级水平，能简单的进行语对话，普通话流利。,1
34554,语 一般 能听说读写,1


In [24]:
job_table = job_table(path)

In [25]:
job_table

,岗位编号,招聘对象代码,招聘对象,招聘职位,对应聘者的专业要求,岗位最低学历,岗位工作地点,岗位工作年限,具体要求,具体要求字数
0,135144,-1.0,NaN,业务员,NaN,2.0,深圳市,0,NaN,NaN
1,137045,-1.0,NaN,电子技术支持工程师,电子信息工程学,1.0,龙岗区,0,要求电子类中专以上文凭，为人诚恳，工作负责，熟悉PROTEL****自动排版系统；对计算机常...,70.0
2,146798,-1.0,NaN,仓管,【工商管理】,1.0,龙岗区,0,电子厂工作经验2年以上（非电子厂勿扰！）；十分熟悉仓库管理流程；懂金碟登帐者优先!,41.0
3,436321,2.0,社会无职,销售代表,NaN,1.0,深圳市,0,女，年龄****-****，高中以上学历，普通话粤语流利，善于沟通，有销售经验者优先。,43.0
4,440725,99.0,不限,造价员,工民建,1.0,深圳市,3,所学专业可以是其他专业，但应有工业与民用建筑土建工程、或给排水工程、或强弱电工程、或通风与空...,109.0
...,...,...,...,...,...,...,...,...,...,...
2099,44263687,1.0,社会在职,跟单文员,NaN,2.0,深圳市,1,女 1名 高中或以上文化，有两年以上丝移印跟单工作经验，字迹端正秀丽，责任心强。,42.0
2100,44274688,3.0,应届毕业,英语播音员,英语,3.0,宝安区,0,男女不限，英语本科以上学历，已过专业英语八级或者普通话二甲，口语好，音质甜美，善于模仿，有播...,66.0
2101,44285651,1.0,社会在职,电气工程师,NaN,NaN,深圳市,5,NaN,NaN
2102,44298679,2.0,社会无职,仓库管理员,NaN,2.0,深圳市,0,1. 1年以上电子厂仓库管理经验2. 熟练金碟登帐3. 能吃苦耐劳。4. 限女性,40.0


In [27]:
job_table.岗位工作年限.value_counts()

 0    1060
 1     432
 3     423
 5     188
-1       1
Name: 岗位工作年限, dtype: int64

In [ ]:
# 最近工作岗位离散
# 最近所在行业离散
#当前工作所在地离散
# 工作年限是否符合
# 专业是否符合
#语言能力：普通话标准、流利、精通、流畅、熟练 二级甲等（二甲、二级甲）、二级乙等（二级乙、二乙）、粤语、四级、六级、八级、日语


In [71]:
person.工作年限.value_counts()

0      13476
8       7159
9       6410
7       5038
10      4961
11      3567
12      2825
13      2127
14      1622
6       1507
15      1279
16      1207
17       687
18       529
19       434
20       321
21       250
22       181
23       175
5        169
24       154
25       121
26       115
27        75
29        57
28        55
30        45
31        41
32        32
34        29
4         24
33        23
35        16
36         9
39         7
37         7
3          7
2          6
45         6
38         6
40         5
1          5
41         4
43         1
116        1
50         1
47         1
46         1
44         1
Name: 工作年限, dtype: int64

In [76]:
(job_table.对应聘者的专业要求.value_counts().rename_axis('对应聘者的专业要求').reset_index(name='总量')).to_csv(r'对应聘者的专业要求.csv')

In [123]:
job_table[job_table.对应聘者的专业要求.apply(lambda x:True if pd.isnull(x) else False)]

,岗位编号,招聘对象代码,招聘对象,招聘职位,对应聘者的专业要求,岗位最低学历,岗位工作地点,岗位工作年限,具体要求,具体要求字数
0,135144,-1.0,NaN,业务员,NaN,2.0,深圳市,0,NaN,NaN
3,436321,2.0,社会无职,销售代表,NaN,1.0,深圳市,0,女，年龄****-****，高中以上学历，普通话粤语流利，善于沟通，有销售经验者优先。,43.0
5,462715,-1.0,NaN,营运经理,NaN,1.0,深圳市,3,有商场管理经验，对商场发生的各种情况有效的解决方法。对商场的业务有计划安排，在各个时期有各方...,84.0
7,489735,-1.0,NaN,财务经理,NaN,NaN,深圳市,3,熟悉商业服务行业小规模纳税企业帐务处理，能独立操作全盘账****公司的帐务问题协调处理，责任...,74.0
9,513553,-1.0,NaN,司机,NaN,NaN,深圳市,0,****-****岁，三年驾龄以上，持B1牌照，能熟练驾驶各种小车，面包车等。品行端庄，吃苦...,58.0
...,...,...,...,...,...,...,...,...,...,...
2098,44260657,1.0,社会在职,海运操作,NaN,1.0,深圳市,3,NaN,NaN
2099,44263687,1.0,社会在职,跟单文员,NaN,2.0,深圳市,1,女 1名 高中或以上文化，有两年以上丝移印跟单工作经验，字迹端正秀丽，责任心强。,42.0
2101,44285651,1.0,社会在职,电气工程师,NaN,NaN,深圳市,5,NaN,NaN
2102,44298679,2.0,社会无职,仓库管理员,NaN,2.0,深圳市,0,1. 1年以上电子厂仓库管理经验2. 熟练金碟登帐3. 能吃苦耐劳。4. 限女性,40.0


In [163]:
#求职者岗位类别次数
# 招聘岗位的投递记录中有多少个不同的岗位类别，招聘岗位的投递记录中有多少个不同的求职者专业
#一份招聘的投递记录中求职者工作年限的mean，max，min，std
#招聘岗位的最低学历要求是否大于求职者的最高学历
#以上已添加

In [164]:
# k-折交叉的标签特征 已添加

In [ ]:
# 以求职者的维度统计信息

In [162]:
求职者文本信息：专业、最近工作岗位、最近所在行业、求职意向岗位、求职意向所在行业、工作经历岗位类别、单位所属行业、专业特长、自荐信、历史工作业绩、
项目经历说明、职责说明、关键技术
岗位的文本信息：招聘岗位名称、要求专业、岗位要求
可以做笛卡尔组合获取匹配值
# 专业特长与岗位要求的匹配值
# 自荐信与岗位要求的匹配值
#历史工作业绩与岗位的匹配值
#岗位类别与专业的匹配值
# 应聘者专业与岗位要求专业匹配值
#等等

In [ ]:
# 投递岗位的名词中是否在工作业绩描述中出现及出现次数及总次数
#投递岗位的名词中是否在项目经历描述中出现及出现次数及总次数